## 3.3 Transformers

In a second experiment. we implemented DistilRoberta as per standard Transformer usage. We passed all the sentences of the WikiBias corpus without any preprocessing through the pretrained model in order to fine-tune.

In [ ]:
import pandas as pd # data processing
from sklearn.model_selection import train_test_split
from transformers import (AutoModelForSequenceClassification,
                          AutoTokenizer,
                          TrainingArguments,
                          Trainer,
                          AutoConfig,
                          DataCollatorWithPadding)
import torch
import torch.nn as nn
from datasets import Dataset, DatasetDict,load_metric
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")
import spacy
nlp_en = spacy.load('en_core_web_lg')
import numpy as np
## Only if your run it in colab 
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
def get_wikibias():
    '''
    We read the wikibias corpus, prepare the partitions and return them. 
    '''
    corpus = pd.read_csv(f'data/wikibias_re.csv')
    #We change to binary classes.
    corpus['label'] = corpus['type'].apply(lambda x: 1 if '1' in x else 0)
    #In order to have some balance in the data, we can remove one sentence from the pairs.
    corpus = corpus[corpus.id.str.contains('\d*_0')]
    #We are left with the data that we need
    corpus = corpus[["sentence", "partition", "label"]]
    #Split
    train = corpus[corpus.partition == 0]
    test =  corpus[corpus.partition == 1]
    val =  corpus[corpus.partition == 2]
    return train[["sentence", "label"]], test[["sentence", "label"]], val[["sentence", "label"]]

train, test, val= get_wikibias()
train.head(10)

,sentence,label
0,""" We "" died of natural causes at age eight in ...",0
2,Evolution is the source of the vast diversity ...,1
4,Credit information such as a persons previous ...,0
6,"Phil Wilayto , former coordinator of A Job is ...",1
8,Most Mellen books are in English ; many are al...,1
10,"It is a distant relative of the medovukha , a ...",0
12,It asks such questions as : How should men int...,1
14,The Disc slot light illuminates briefly when t...,0
16,"In 1953 , the Nehru government bowed to intens...",1
18,Note : Almost every place of interest listed a...,0


In [ ]:
len(train[train.label == 0]), len(train[train.label == 1])

(538, 1975)

## SG2

In [ ]:
#SG2
en_corpus_SG2 = pd.read_csv('data/final_labels_SG2.csv', delimiter=';')
en_corpus_SG2['label'] = en_corpus_SG2['label_bias'].apply(lambda x: 1 if 'Biased' in x else 0)
en_corpus_SG2 = en_corpus_SG2[['text','label']]
en_corpus_SG2 = en_corpus_SG2.rename(columns={'text': 'sentence'})
_, test_SG2 = train_test_split(en_corpus_SG2, train_size=.8, random_state=0)
test_SG2.head(10)

,sentence,label
253,American Outdoor Brands Corp AOBC.O said on Th...,0
541,Black Lives Matter Philadelphia organizer prop...,0
302,And if we look at a subset of guns which the l...,1
3575,While Pence and the president have tried to pa...,1
1698,Many people in politics and media are still no...,1
3048,Their work came on the heels of an IPCC report...,0
3653,"Yet bizarrely, the entirety of the Establishme...",1
841,"Despite a clear margin of victory for Biden, T...",0
3530,When greed and power are exercised by giant mu...,1
473,"Because he just can't help himself, on the Sun...",0


## Checkthat

In [ ]:
#Checkthat
test_Check = pd.read_csv('data/dev_en.tsv', sep='\t')
test_Check['label'] = np.where(test_Check['label']=='SUBJ',1,0)
test_Check = test_Check[['sentence','label']]
test_Check.head(10)

,sentence,label
0,Who will redistribute the hoarded wealth that ...,1
1,What we don’t need is the indiscriminate influ...,1
2,The Social Distance Between Us shows every sig...,0
3,"History shows that McCarthy and McConnell, lik...",0
4,So while it’s not hard to reach a banal point ...,1
5,We can never forget that when the pressure was...,0
6,"Over the past few decades, very few real fight...",1
7,"I just find it a ridiculous argument,” said St...",0
8,"Meanwhile, the radical political project of ma...",1
9,Their posh communities don’t have to deal with...,1


## Transformer

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["Texts"], truncation=True)

datasets = DatasetDict({
    "train": Dataset.from_pandas(train.rename(columns={'sentence': 'Texts', 'label': 'labels'})),
    "dev" : Dataset.from_pandas(val.rename(columns={'sentence': 'Texts', 'label': 'labels'})),
    "test": Dataset.from_pandas(test.rename(columns={'sentence': 'Texts', 'label': 'labels'})),
    "test_SG2": Dataset.from_pandas(test_SG2.rename(columns={'sentence': 'Texts', 'label': 'labels'})),
    "test_Check": Dataset.from_pandas(test_Check.rename(columns={'sentence': 'Texts', 'label': 'labels'})),
    })

tokenized_datasets = datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


tokenized_datasets["train"].set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_datasets["dev"].set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_datasets["test"].set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_datasets["test_SG2"].set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_datasets["test_Check"].set_format("torch", columns=["input_ids", "attention_mask", "labels"])

### Weighted Cross Entropy Loss

In [ ]:
class_weights = (1 - (train["label"].value_counts().sort_index() / len(train))).values
class_weights

array([0.78591325, 0.21408675])

In [ ]:
class_weights = torch.from_numpy(class_weights).float().to("cuda")
print(class_weights)

class WeightedLossTrainer(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False):

        outputs = model(**inputs)
        logits = outputs.get("logits")
        labels = inputs.get("labels")

        loss_func = nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_func(logits, labels)

        return (loss, outputs) if return_outputs else loss

tensor([0.7859, 0.2141], device='cuda:0')


### Train with WIKIBIAS

In [ ]:
def compute_metrics(eval_pred):
    metric1 = load_metric("precision", trust_remote_code=True)
    metric2 = load_metric("recall", trust_remote_code=True)
    metric3 = load_metric("f1", trust_remote_code=True)
    metric4 = load_metric("accuracy", trust_remote_code=True)

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    precision = metric1.compute(predictions=predictions, references=labels, average="macro")["precision"]
    recall = metric2.compute(predictions=predictions, references=labels, average="macro")["recall"]
    f1 = metric3.compute(predictions=predictions, references=labels, average="macro")["f1"]
    accuracy = metric4.compute(predictions=predictions, references=labels)["accuracy"]

    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}

In [ ]:
# Load and compile our model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config = AutoConfig.from_pretrained("distilroberta-base", num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained("distilroberta-base",config=config).to(device)

training_args = TrainingArguments(output_dir="test_trainer",
                                  evaluation_strategy='epoch',
                                  learning_rate=3e-5,
                                  load_best_model_at_end=True,
                                  save_strategy="epoch",
                                  num_train_epochs=3)

trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)
trainer.train()


In [ ]:
preds_output_test = trainer.predict(tokenized_datasets["test"])
preds_output_test.metrics

### SG2

In [ ]:
preds_output_SG2 = trainer.predict(tokenized_datasets["test_SG2"])
preds_output_SG2.metrics

### CheckThat!

In [ ]:
preds_output_Check = trainer.predict(tokenized_datasets["test_Check"])
preds_output_Check.metrics